In [3]:
import requests as r

In [4]:
url = 'https://api.covid19api.com/dayone/country/brazil'
resp = r.get(url)

In [ ]:
resp.status_code

In [6]:
raw_data = resp.json()

In [ ]:
raw_data[0]

In [8]:
final_data = []
for obs in raw_data:
    final_data.append([obs['Confirmed'],obs['Deaths'],obs['Recovered'],obs['Active'],obs['Date']])

In [ ]:
final_data.insert(0, ['Confirmados', 'Obitos', 'Recuperados', 'Ativos', 'Data'])
final_data

In [ ]:
final_data

In [11]:
CONFIRMADOS = 0
OBITOS = 1
RECUPERADOS = 2
ATIVOS = 3
DATA = 4

In [ ]:
for i in range(1, len(final_data)):
    final_data[i][DATA] = final_data[i][DATA][:10]
final_data    

In [13]:
import datetime as dt

In [14]:
import csv

In [15]:
with open('brasil-covid.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerows(final_data)

In [16]:
for i in range(1, len(final_data)):
    final_data[i][DATA] = dt.datetime.strptime(final_data[i][DATA], '%Y-%m-%d')

In [ ]:
final_data

In [18]:
def get_datasets(y, labels):
    if type(y[0]) == list:
        datasets = []
        for i in range(len(y)):
            datasets.append({
                'label': labels[i],
                'data': y[i]
                })
        return datasets
    else:
        return [
            {
                'label':labels[0],
                'data': y
            }
        ]            

In [19]:
def set_title(title=''):
    if title != '':
        display = 'true'
    else:
        display = 'false'
    return {
        'title': title,
        'display': display
    }

In [20]:
def create_chart(x, y, labels, kind='bar', title=''):
    
    datasets = get_datasets(y, labels)
    options = set_title(title)
    
    chart = {
        'type' : kind,
        'data' : {
            'labels': x,
            'datasets' : datasets
        },
        'options' : options
    }
    
    return chart

In [21]:
def get_api_chart(chart):
    url_base = 'https://quickchart.io/chart'
    resp = r.get(f'{url_base}?c={str(chart)}')
    return resp.content

In [22]:
def save_image(path, content):
    with open(path, 'wb') as image:
        image.write(content)

In [ ]:
!pip install Pillow

In [24]:
import IPython.display as display
from PIL import Image

In [ ]:
y_data_1 = []
for obs in final_data[1::30]:
    y_data_1.append(obs[CONFIRMADOS])

y_data_2 = []
for obs in final_data[1::30]:
    y_data_2.append(obs[RECUPERADOS])
    
labels = ['Confirmados', 'Recuperados']

x = []
for obs in final_data[1::30]:
    x.append(obs[DATA].strftime('%d/%m/%Y'))
    
chart = create_chart(x, [y_data_1, y_data_2], labels, title='Gráfico confirmados vs recuperados')
chart_content = get_api_chart(chart)
save_image('meu-primeiro-grafico.png', chart_content)
display.display(Image.open('meu-primeiro-grafico.png'))

In [26]:
from urllib.parse import quote

In [27]:
def get_api_qrcode(link):
    text = quote(link) # parsing do link para url
    url_base = 'https://quickchart.io/qr'
    resp = r.get(f'{url_base}?text={text}')
    return resp.content

In [ ]:
url_base = 'https://quickchart.io/chart'
link = f'{url_base}?c={str(chart)}'
save_image('qr-code.png', get_api_qrcode(link))
display.display(Image.open('qr-code.png'))